In [161]:
import requests
import sys
from bs4 import BeautifulSoup as bs
import time
import math
import pandas as pd

In [162]:
search_word = "sony digital camera"
#payload = {'q':search_word}
payload = {'_nkw':search_word}
    
r = requests.get('https://www.ebay.com/sch/i.html', params = payload)
page = r.content
soup = bs(page, "html.parser")

# Count the number of pages of the search result
num_items = soup.find_all("span", class_ = "rcnt")
num_items = get_text_from_elements(num_items)
num_items = int(num_items[-1].replace(",","")) # replace "1,234" into "1234" and convert into integer

items_per_page = 50
link_pages = int(math.ceil(num_items / items_per_page))

print("The number of pages is", link_pages)

The number of pages is 220


In [22]:
#products_table_df = pd.DataFrame(columns = ["Name", "Ratings", "Price"])

In [163]:
# Scrape the data from each search result page using for loop

# Create empty lists
products_table = []

# For each search result page, get all the review rating, title, and price data
for i, page in enumerate(range(min(link_pages, 50))): # to avoid kernel dead, limit maximum pages (50 items * 50 pages)
    page_num = i + 1
    payload = {'_nkw':search_word, '_pgn': page_num}
    r = requests.get('https://www.ebay.com/sch/i.html', params = payload)
    page = r.content
    soup = bs(page, "html.parser")

    # Get product data and store in a list of "products"
    result_items = soup.find_all('div', id = "ResultSetItems")
    r_lists = range(2, items_per_page + 1)
    products = result_items[0].find_all("li", r = "1")
    for r_list in r_lists:
        product = result_items[0].find_all("li", r = str(r_list))[0] # To make class bs4.element.ResultSet into bs4.element.Tag
        products.append(product)

    
    for product in products:
        # Remove products which have not review ratings
        if len(product.find_all("div", class_ = "star-ratings")) != 0:
            # Get review rating data
            review = float(product.find('div', class_ = "star-ratings").find("a")["aria-label"].split(" ")[1])
            # Get title data
            title = product.find('h3', class_ = "lvtitle").find("a").text
            # Get Price data
            price = float(product.find('li', class_ = "lvprice prc").find("span").text.split("$")[-1].replace(",",""))
            # Merge the data
            product_table = [title, review, price]
            # Mutate the data table
            products_table.append(product_table)

            
print(products_table)
print("The number of products is", len(products_table))

[['\r\n\t\t\t\tSony Alpha a7R III Mirrorless Digital Camera (Body Only)\n', 4.87, 2849.99], ['Sony Alpha a6000 Mirrorless Digital Camera with 16-50mm Lens Black', 4.85, 579.95], ['\r\n\t\t\t\tSony Alpha a5000 Mirrorless Digital Camera with 16-50mm Lens (Black)\n', 4.81, 378.42], ['Sony Alpha a5100 Mirrorless Digital Camera with 16-50mm Lens (Black) PRO KIT NEW', 4.84, 524.99], ['New listing\r\n\t\tSony Alpha a6000 24.3MP Digital Camera - Black (Body Only) (low reserve)', 4.93, 299.99], ['\r\n\t\t\t\tSony Alpha a6300 Mirrorless Digital Camera w/16-50mm Lens\n', 4.86, 897.03], ['Sony a5000 Digital Camera with 16-50mm Lens OSS Black ILCE5000L/B', 4.81, 335.0], ['Sony Alpha a5100 24.3MP Digital Camera - Black (Body Only)', 4.9, 309.99], ['New listing\r\n\t\tSony Cyber-shot DSC-HX400V 20.4MP Digital Camera - Black', 4.81, 349.99], ['Sony Cyber-shot DSC-W350 14.1MP Digital Camera - Black', 4.78, 29.99], ['Sony Alpha a5000 Mirrorless Digital Camera with 16-50mm Lens (Black)', 4.81, 329.99], [

In [164]:
# Uniqueness check using 'dict'
product_name = {}
count = 0
for product in products_table:
    if product[0] not in product_name:
        product_name[product[0]] = 1
    else:
        product_name[product[0]] += 1   
print("The number of unique products is", len(product_name), "out of", len(products_table))

The number of unique products is 1359 out of 1632


In [165]:
# Convert into Pandas Data frame
import pandas as pd
products_table_df = pd.DataFrame(products_table, columns = ["Name", "Ratings", "Price"])
products_table_df

,Name,Ratings,Price
0,\r\n\t\t\t\tSony Alpha a7R III Mirrorless Digi...,4.87,2849.99
1,Sony Alpha a6000 Mirrorless Digital Camera wit...,4.85,579.95
2,\r\n\t\t\t\tSony Alpha a5000 Mirrorless Digita...,4.81,378.42
3,Sony Alpha a5100 Mirrorless Digital Camera wit...,4.84,524.99
4,New listing\r\n\t\tSony Alpha a6000 24.3MP Dig...,4.93,299.99
5,\r\n\t\t\t\tSony Alpha a6300 Mirrorless Digita...,4.86,897.03
6,Sony a5000 Digital Camera with 16-50mm Lens OS...,4.81,335.00
7,Sony Alpha a5100 24.3MP Digital Camera - Black...,4.90,309.99
8,New listing\r\n\t\tSony Cyber-shot DSC-HX400V ...,4.81,349.99
9,Sony Cyber-shot DSC-W350 14.1MP Digital Camera...,4.78,29.99


In [189]:
# Output to CSV file as raw data
products_table_df.to_csv("ebay_raw.csv")

In [187]:
# Data Cleaning
# Remove duplicated products
products_table_df2 = products_table_df.drop_duplicates("Name")

# Delele "'\r', '\n', '\t'" from Name column
names = [name.replace("\t", "") for name in products_table_df2["Name"]]
names = [name.replace("\r", "") for name in names]
names = [name.replace("\n", "") for name in names]
names = pd.DataFrame(names)
products_table_df2["Name"] = names
products_table_df2 = products_table_df2[-products_table_df2["Name"].isnull()] # remove rows with NaN name

# Sort by product name and reset index
products_table_df2 = products_table_df2.sort_values(by="Name")
products_table_df2 = products_table_df2.reset_index(drop=True)

C:\Users\hika1\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [188]:
products_table_df2

,Name,Ratings,Price
0,# White SONY Digital Camera A6000 ILCE-6000 w/...,4.85,395.00
1,(RI2) Sony Cyber-shot DSC-H300 20.1MP Digital ...,4.53,109.99
2,**BRAND NEW** Sony Cyber-shot DSC-H2 6.0MP Dig...,5.00,4069.00
3,*BRAND NEW* Sony Cyber-shot DSC-RX100 Digital ...,4.86,508.33
4,*Working* Sony Alpha DSLR-A100 10.2MP w/ Two L...,4.87,190.00
5,- Sony Alpha a700 12.2MP Digital SLR Camera wi...,4.79,550.00
6,2Sony Cyber-shot DSC-W830 20.1MP Digital Camer...,4.95,789.00
7,5x Sony MVC-FD200 Mavica Digital Camera - Grade C,5.00,350.69
8,AS IS: Sony Cyber-shot DSC-HX60V 20.4MP Digita...,4.85,499.95
9,BUNDLE Sony Cyber-shot DSC-HX20V 18.2 MP Exmor...,4.71,349.99


In [190]:
# Output to CSV file as processed data
products_table_df2.to_csv("ebay_clean.csv")

In [ ]:
##############################################
####### Below code is for testing ############

In [104]:
    page_num = 30
    payload = {'_nkw':search_word, '_pgn': page_num}
    r = requests.get('https://www.ebay.com/sch/i.html', params = payload)
    page = r.content
    soup = bs(page, "html.parser")
    result_items = soup.find_all('div', id = "ResultSetItems")
    
    r_lists = range(2,51)
    products = result_items[0].find_all("li", r = "1")
    for r_list in r_lists:
        product = result_items[0].find_all("li", r = str(r_list))[0]
        products.append(product)
    products

[<li _sp="p2045573.m1686.l8" class="sresult lvresult clearfix li" id="item3f974c9a62" listingid="273121319522" r="1">
 <div class="lvpic pic img left" iid="273121319522">
 <div class="lvpicinner full-width picW">
 <a class="img imgWr2" href="https://www.ebay.com/itm/Sony-a5000-20-1MP-Digital-Camera-55-210mm-f-4-5-6-3-OSS-Lens-Silver-White/273121319522?hash=item3f974c9a62:g:3icAAOSwQUhaslto">
 <img alt="Sony a5000 20.1MP Digital Camera + 55-210mm f/4.5-6.3 OSS Lens - Silver/White" class="img" src="https://i.ebayimg.com/thumbs/images/g/3icAAOSwQUhaslto/s-l225.jpg"/>
 </a>
 </div></div>
 <h3 class="lvtitle"><a class="vip" href="https://www.ebay.com/itm/Sony-a5000-20-1MP-Digital-Camera-55-210mm-f-4-5-6-3-OSS-Lens-Silver-White/273121319522?hash=item3f974c9a62:g:3icAAOSwQUhaslto" title="Click this link to access Sony a5000 20.1MP Digital Camera + 55-210mm f/4.5-6.3 OSS Lens - Silver/White">Sony a5000 20.1MP Digital Camera + 55-210mm f/4.5-6.3 OSS Lens - Silver/White</a>
 </h3>
 <div class="l

In [32]:
float(products[0].find('div', class_ = "star-ratings").find("a")["aria-label"].split(" ")[1])

products[0].find('h3', class_ = "lvtitle").find("a").text

float(products[0].find('li', class_ = "lvprice prc").find("span").text.split("$")[-1])

360.0

In [ ]:
# From lab 3
# (Uses list comprehension to parse out the cleaned text strings from a list of elements returned from a BeautifulSoup selection.)
def get_text_from_elements(elements):
    return [e.text.strip() for e in elements]